# Mesh example
### Example of how to use the mesh functionality in JuAFEM using a Taylor-Hood element for incompressible elasticity

In [1]:
include("../src/JuAFEM.jl")
using JuAFEM

In [2]:
function main()
    # Set up a mesh
    xs = [0.0, 0.0] # Start coordinate
    xe = [2.0, 2.0] # End coordinate
    nel = [10,10] # Number of elements in each dir.
    
    # Generate a mesh object
    fe_m = FEMesh(Dim{2},JuAFEM.Triangle(),xs,xe,nel)
    
    # Set up two fields with different FE approximation order
    # Quadratic displacement
    u_fs = Lagrange{2, JuAFEM.Triangle, 2}()
    u_qr = get_gaussrule(Dim{2}, JuAFEM.Triangle(), 2)
    u_fe_v = FEValues(Float64, u_qr, u_fs)
    u_field = FEField(u_fs,2,"displacement")
    
    # Linear pressure
    p_fs = Lagrange{2, JuAFEM.Triangle, 1}()
    p_qr = get_gaussrule(Dim{2}, JuAFEM.Triangle(), 2)
    p_fe_v = FEValues(Float64, p_qr, p_fs)
    p_field = FEField(p_fs,1,"pressure")
    
    fields = [u_field,p_field]
    
    # Generate a FEDofs object
    fe_d = FEDofs(fe_m,fields)
    return fe_d
    
    # Material parameters
    G = 10.0 # Shear modulus
    
    
    # Since this problem is linear we can just assemble the stiffness matrix and solve
    _K = start_assemble() # Assembler
    
    # Initializing stuff
    Ke = zeros(15,15)
    D = G * diagm([2.0,2.0,1.0]) # Material stiffness
    Bϵ = zeros(3,12)
    B_div = zeros(1,12)
    B_dev = zeros(3,12)
    
    for el in 1:get_number_of_elements(fe_m)
        # Get node coordinates for first field, and reinitiate its fe_values
        x_u = get_node_coordinates(fe_d,el,1)
        reinit!(u_fe_v,x_u)
        
        # Get node coordinates for second field, and reinitiate its fe_values
        x_p = get_node_coordinates(fe_d,el,2)
        reinit!(p_fe_v,x_p)
        
        m = get_element_dofs(fe_d, el) # Global element dofs
        m_u = get_element_field_dofs(fe_d,el,1) # Global element field dofs
        m_p = get_element_field_dofs(fe_d,el,2)
        
        fill!(Ke,0.0) # Reset
        for q_point in 1:length(JuAFEM.points(get_quadrule(u_fe_v)))
            dNdxT =  shape_gradient(u_fe_v,q_point)
            NT = shape_value(p_fe_v,q_point)
            dNdx = reinterpret(Float64,dNdxT,(2,6))
            N = reinterpret(Float64,NT,(length(NT),))

            Bϵ[[1,3],1:2:11] = dNdx
            Bϵ[[3,2],2:2:12] = dNdx
            B_div[:] = dNdx[:]
            B_dev[:] = Bϵ - 1/3 * [1.0,1.0,0.0] * B_div
            
            
            S = Bϵ' * D * B_dev * detJdV(u_fe_v,q_point)
            C = B_div' * N' * detJdV(u_fe_v,q_point)
            
            Ke[1:12,1:12] += S
            Ke[1:12,13:15] -= C
            Ke[13:15,1:12] -= C'
        end
        println(size(Ke))
        println(size(m))
        assemble(m,_K,Ke) # Assemble element stiffness
    end
    K = end_assemble(_K)
    return K
    a,fb = solveq(K,f,bc)
    
    # Write to VTK
end

main (generic function with 1 method)

In [3]:
out = main()

JuAFEM.FEDofs(JuAFEM.FEMesh{2,Float64}([[0.0,0.0],[0.2,0.0],[0.4,0.0],[0.6,0.0],[0.8,0.0],[1.0,0.0],[1.2,0.0],[1.4,0.0],[1.6,0.0],[1.8,0.0]  …  [0.2,2.0],[0.4,2.0],[0.6,2.0],[0.8,2.0],[1.0,2.0],[1.2,2.0],[1.4,2.0],[1.6,2.0],[1.8,2.0],[2.0,2.0]],3x200 Array{Int64,2}:
  1   2   2   3   3   4   4   5   5   6  …  107  107  108  108  109  109  110
  2  13   3  14   4  15   5  16   6  17     118  108  119  109  120  110  121
 12  12  13  13  14  14  15  15  16  16     117  118  118  119  119  120  120,Array{T,1}[[JuAFEM.FEBoundary{1}(2x10 Array{Int64,2}:
 1  2  3  4  5  6  7  8   9  10
 2  3  4  5  6  7  8  9  10  11),JuAFEM.FEBoundary{1}(2x10 Array{Int64,2}:
 11  22  33  44  55  66  77  88   99  110
 22  33  44  55  66  77  88  99  110  121),JuAFEM.FEBoundary{1}(2x10 Array{Int64,2}:
 111  112  113  114  115  116  117  118  119  120
 112  113  114  115  116  117  118  119  120  121),JuAFEM.FEBoundary{1}(2x10 Array{Int64,2}:
  1  12  23  34  45  56  67  78   89  100
 12  23  34  45  56  67  7

In [4]:
out.fieldindex

2-element Array{Array{Int64,1},1}:
 [1,2,4,5,6,7,9,10,11,12,14,15]
 [3,8,13]                      